In [1]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
import pandas

train = pandas.read_csv(r'C:\Users\jyellin\re_1\tacred\results\general\train-ud_paths_per_relation_no_relation.csv')
test = pandas.read_csv(r'C:\Users\jyellin\re_1\tacred\results\general\test-ud_paths_per_relation_no_relation.csv')
    

In [3]:
def transform_row(r):
    return '{entity1} {org_path} {entity2}'.format(entity1=r.type1[0:3], org_path=r.path, entity2=r.type2[0:3])

train['path'] = train.apply(transform_row, axis=1)
test['path'] = test.apply(transform_row, axis=1)

In [4]:
relation_to_paths = train.groupby(['relation'])['path'].unique().to_dict()

In [5]:
def label_sentence_recall(test_row, relation):
    path = test_row['path']
    true_relation = test_row['relation']
    
    if path in relation_to_paths[relation]:
        if true_relation == relation:
            return 'TP'
        else:
            return 'FP'
    else:
        return 'N'

In [6]:
for relation in filter(lambda relation: relation != 'no_relation', relation_to_paths.keys()):
    test[relation] = test.apply(lambda row : label_sentence_recall(row, relation), axis=1)    

In [7]:
stats = []
overall_tp = 0
overall_fp = 0

for relation in filter(lambda relation: relation != 'no_relation', relation_to_paths.keys()):

    counts = test[relation].value_counts().to_dict()
        
    tp = counts.get('TP', 0)
    fp = counts.get('FP', 0)
    precision = tp / (tp+fp)

    stats.append( (relation, round(precision,4) ) )
    
    overall_tp =+ tp
    overall_fp =+ fp
    
overall_precision = tp / (tp+fp)

    
#matched = sum(test_total_by_r_and_p[test_total_by_r_and_p['total-for-relation']>0]['total'].to_list())
#matched_lax = sum(test_total_by_r_and_p[test_total_by_r_and_p['total-across-relations']>0]['total'].to_list())
#total = sum(test_total_by_r_and_p['total'].to_list())

stats.append( ('overall', round(overall_precision,4) ) )
stats = sorted(stats, key=lambda x: x[1], reverse=True)

In [8]:
import plotly.graph_objs as go

overall_index = next(i for (i, (relation, score)) in enumerate(stats) if relation == 'overall')

colors = ['rgb(58,200,225)'] * len(stats)
colors[overall_index] = 'rgb(181,59,89)'


trace1 = go.Bar(
    x=[stat[0] for stat in stats],
    y=[stat[1] for stat in stats],
    name='Precision',
    width=0.5,
    marker=dict(
        color=colors,
        line=dict(
            color='rgb(8,48,107)',
            width=0.5),
        ),
    opacity=0.6
)

data = [trace1]
layout = go.Layout(
    title='UD Path - Precision',
    xaxis_title="Relation",
    yaxis_title="Precision",        
    barmode='overlay',
    width=1200,
    xaxis = go.layout.XAxis(
        tickangle = 45,
        automargin = True
        
    )
    
        
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)